In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("covid_data_196.csv")

In [3]:
xvar = ["Employed", "Males", "Vulnerable_pop", "Health_Insurance", 
        "Secondary_Education", "Life_Expectancy", "prev_cronic_prov", 
        "Indice_Pobreza_Compuesto", "White_1000_Inhab", "Assian_1000_Inhab", 
        "Black_1000_Inhab", "prev_diferencial_prov_endes", "prev_hipertension_prov_endes", 
        "prev_diabetes_prov_endes", "prev_obesidad_prov_endes", "Days_Till_Attended", 
        "SD_Days_Till_Attended",  "Travel_Time_toHFacility_Hours", "SD_TTtHFH",
        "Waiting_Time_4Attention_Hours", "SD_WT4AH", "logPD_1000", "Overcrowding",
        "Natural_Region1", "Natural_Region2", "Natural_Region3"]

In [4]:
datac = data.dropna()

x = datac[xvar]
Y = datac["logmuertes1000"]

In [14]:
datac

,_ID,_CX,_CY,IDDPTO,DEPARTAMEN,prov,PROVINCIA,CAPITAL,FUENTE,dep,...,casos_1000_Inhab,muertes_1000_Inhab,logcasos1000,logmuertes1000,dominio,Natural_Region,Natural_Region1,Natural_Region2,Natural_Region3,logPD_1000
0,1,-77.773167,-6.437059,1,AMAZONAS,101,CHACHAPOYAS,CHACHAPOYAS,INEI,1,...,37.995892,0.414370,3.637478,-0.880997,sierra norte,sierra,0,1,0,0.614774
1,2,-78.402692,-5.087450,1,AMAZONAS,102,BAGUA,BAGUA,INEI,1,...,108.002700,1.322537,4.682156,0.279552,selva,selva,0,0,1,1.538134
2,3,-77.873091,-5.683441,1,AMAZONAS,103,BONGARA,JUMBILLA,INEI,1,...,14.393260,0.390061,2.666760,-0.941452,selva,selva,0,0,1,2.541240
3,4,-78.038591,-4.166722,1,AMAZONAS,104,CONDORCANQUI,SANTA MARIA DE NIEVA,INEI,1,...,78.714386,0.518013,4.365826,-0.657755,selva,selva,0,0,1,-1.905125
4,5,-78.077829,-6.323335,1,AMAZONAS,105,LUYA,LAMUD,INEI,1,...,9.924386,0.247547,2.294995,-1.396155,sierra norte,sierra,0,1,0,0.998217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,191,-80.740769,-3.969757,24,TUMBES,2402,CONTRALMIRANTE VILLAR,ZORRITOS,INEI,24,...,22.177898,1.282234,3.099096,0.248604,costa norte,costa,1,0,0,0.231815
191,192,-80.256500,-3.652287,24,TUMBES,2403,ZARUMILLA,ZARUMILLA,INEI,24,...,31.876997,1.289821,3.461885,0.254503,costa norte,costa,1,0,0,0.147406
192,193,-74.058177,-8.672964,25,UCAYALI,2501,CORONEL PORTILLO,PUCALLPA,INEI,25,...,40.443245,0.739260,3.699900,-0.302106,selva,selva,0,0,1,0.893040
193,194,-73.218893,-10.389299,25,UCAYALI,2502,ATALAYA,ATALAYA,INEI,25,...,9.427460,0.141919,2.243627,-1.952501,selva,selva,0,0,1,0.625074


In [5]:
#use of fuel lighting
#posgraduated
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn.model_selection import RepeatedKFold


# Define cross-validation method
cv = RepeatedKFold(n_splits=4, n_repeats=10,random_state=0)

# Define grid
param_grid = {
    'alpha': [1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0.1,1.0,0.01)
}

# Define evaluation post-processing criteria
from sklearn.metrics import mean_squared_error

scoring = make_scorer(mean_squared_error)

from sklearn.linear_model import ElasticNet
#model_LR = LogisticRegression(random_state=0, penalty='l2', solver='liblinear')

model = ElasticNet(random_state=0, fit_intercept=True)

# Define search
search_ddnn = GridSearchCV(estimator = model, param_grid=param_grid, scoring='neg_mean_squared_error',
                           cv=cv, n_jobs=-1, verbose = 4)

# Perform the search
results_ENLR = search_ddnn.fit(x, Y)

Fitting 40 folds for each of 450 candidates, totalling 18000 fits


In [6]:
print('MSE: %.3f' % results_ENLR.best_score_)
print('Config: %s' % results_ENLR.best_params_)

MSE: -0.296
Config: {'alpha': 1.0, 'l1_ratio': 0.20999999999999996}


In [7]:
model_ENLR = ElasticNet(random_state=0, fit_intercept=True, alpha = 1, l1_ratio= 0.20999999999999996)
model_ENLR.fit(x, Y)

ElasticNet(alpha=1, l1_ratio=0.20999999999999996, random_state=0)

In [8]:
Results_LR = pd.DataFrame(list(zip(x.columns.values,model_ENLR.coef_)), columns =["Variable", "Coefficient"])
Results_LR

,Variable,Coefficient
0,Employed,6.798042e-04
1,Males,0.000000e+00
2,Vulnerable_pop,3.716705e-03
3,Health_Insurance,5.945609e-04
4,Secondary_Education,-4.390458e-08
5,Life_Expectancy,0.000000e+00
6,prev_cronic_prov,-4.208333e-06
7,Indice_Pobreza_Compuesto,-4.497059e-02
8,White_1000_Inhab,-3.098146e-03
9,Assian_1000_Inhab,0.000000e+00


In [9]:
import sklearn
print(sklearn.metrics.SCORERS.keys())

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weig

In [13]:
model_ENLR.predict(x.iloc[0:1,:])

array([-0.37721619])